# Initialize PyImageJ

In [1]:
# Import module
import jpype
# Enable Java imports
import jpype.imports
# Pull in types
from jpype.types import *

import scyjava as sj # scyjava : Supercharged Java access from Python, see https://github.com/scijava/scyjava
import imagej

# Configurations
# NOTE: The ImageJ2 gateway is initialized through a Java Virtual Machine (JVM). If you want to configure the JVM, it must be done before initializing an ImageJ2 gateway.
# sj.config.add_option('-Xmx10g') # adjust memory available to Java
sj.config.endpoints.append('ome:formats-gpl:6.11.1')

In [2]:
Fiji_Local = r"C:\Users\confocal_microscope\Desktop\Tools\Fiji.app"

# ij = imagej.init(Fiji_Local) # Same as "ij = imagej.init(Fiji_Local, mode='headless')", PyImageJ’s default mode is headless
# ij = imagej.init(Fiji_Local, mode='gui') # GUI mode (會卡在這一行 -> blocking), for more explainations : https://pyimagej.readthedocs.io/en/latest/Initialization.html#gui-mode
ij = imagej.init(Fiji_Local, mode='interactive') # Interactive mode (可以繼續向下執行 -> non-blocking), for more explainations : https://pyimagej.readthedocs.io/en/latest/Initialization.html#interactive-mode
ij.ui().showUI() # display the Fiji GUI

print(ij.getApp().getInfo(True)) # ImageJ2 2.9.0/1.54b

ImageJ2 2.9.0/1.54b; Java 1.8.0_362 [amd64]; 196MB of 14542MB


# Import Customized modules and Create ImageJ Object

In [3]:
import os
import sys
import re
from typing import TextIO
from datetime import datetime

from glob import glob
import logging
from skimage.io import imread, imshow, imsave


sys.path.append(r"C:\Users\confocal_microscope\Desktop\ZebraFish_AP_POS\modules") # add path to scan customized module

from logger import init_logger
from fileop import create_new_dir
from composite_ij import dump_info, median_R1_and_mean3D_R2

In [4]:
# Bio-Format Reader
loci = jpype.JPackage("loci")
loci.common.DebugTools.setRootLevel("ERROR")
Reader = loci.formats.ImageReader()

# [IMPORTANT] some ImageJ plugins need to be new before use
from ij.plugin.frame import RoiManager
from ij.plugin import ImageCalculator
from ij.plugin import ChannelSplitter
from ij.plugin import ZProjector
from ij.plugin import RGBStackMerge
from ij.plugin import RGBStackConverter

rm = RoiManager()
imageCalculator = ImageCalculator()
channelSplitter = ChannelSplitter()
zProjector = ZProjector()
rgbStackMerge = RGBStackMerge()
rgbStackConverter = RGBStackConverter()

In [5]:
log = init_logger(r"ImageJ")

# Start Process